## Dépendances

In [24]:
import urllib
import bs4
import pandas
from urllib import request
import numpy as np
import re

In [2]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wikipedia
from wikipedia import WikipediaPage
wikipedia.set_lang("fr")

In [3]:
cam = WikipediaPage('Albert Camus').summary
cam

"Albert Camus, né le 7 novembre 1913 à Mondovi (aujourd’hui Dréan), en Algérie, et mort accidentellement le 4 janvier 1960 à Villeblevin, est un écrivain, philosophe, romancier, dramaturge, essayiste et nouvelliste français. Il est aussi journaliste militant engagé dans la Résistance française et, proche des courants libertaires, dans les combats moraux de l'après-guerre.\nSon œuvre comprend des pièces de théâtre, des romans, des nouvelles, des films, des poèmes et des essais dans lesquels il développe un humanisme fondé sur la prise de conscience de l'absurde de la condition humaine mais aussi sur la révolte comme réponse à l'absurde, révolte qui conduit à l'action et donne un sens au monde et à l'existence. Il reçoit le prix Nobel de littérature en 1957.\nDans le journal Combat, il prend position aussi bien sur la question de l'indépendance de l'Algérie que sur ses rapports avec le Parti communiste algérien, qu'il quitte après un court passage de deux ans. Il proteste successivement 

## Scraping

### Liste d'auteurs

On charge la [page d'intérêt](https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_alphab%C3%A9tique) pour scraper la liste des écrivains français depuis Wikipédia.

In [4]:
base_url="https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_alphab%C3%A9tique"
request_text= request.urlopen(base_url).read()
page = bs4.BeautifulSoup(request_text, "lxml") #qui est bien une page html maintenant 

In [5]:
# == Obtenir toutes les catégories par lettre ==
fond=page.find("div",{"class":"mw-parser-output"}).findAll("ul")

fond[0]

<ul><li><a href="/wiki/Fran%C3%A7ois-Jean_Willemain_d%27Abancourt" title="François-Jean Willemain d'Abancourt">François-Jean Willemain d'Abancourt</a> (<a href="/wiki/1745" title="1745">1745</a> - <a href="/wiki/1803" title="1803">1803</a>)</li>
<li><a href="/wiki/Jacques_Abbadie" title="Jacques Abbadie">Jacques Abbadie</a> (<a href="/wiki/1654" title="1654">1654</a> - <a href="/wiki/1727" title="1727">1727</a>)</li>
<li><a href="/wiki/%C3%89liette_Ab%C3%A9cassis" title="Éliette Abécassis">Éliette Abécassis</a> (<a href="/wiki/1969" title="1969">1969</a> - )</li>
<li><a href="/wiki/Pierre_Ab%C3%A9lard" title="Pierre Abélard">Pierre Abélard</a> (<a href="/wiki/1079" title="1079">1079</a>-<a href="/wiki/1142" title="1142">1142</a>)</li>
<li><a href="/wiki/Raymond_Abellio" title="Raymond Abellio">Raymond Abellio</a> (<a href="/wiki/1907" title="1907">1907</a> - <a href="/wiki/1986" title="1986">1986</a>)</li>
<li><a href="/wiki/Edmond_About" title="Edmond About">Edmond About</a> (<a href=

In [ ]:
""""
fond[0].findAll("li")[0].find("a").get("title") #ça marche

ltest=fond[0].findAll("li")
listee=[]
for li in ltest:
    listee.append(li.find("a").get("title"))
listee
"""

On va créer une liste *auteurs1* composée de listes d'auteurs par ordre alphabétique (nom de famille).

In [31]:
# == Obtenir les auteurs pour tous les bloc précédents ==
auteurs1=[]
for ul in fond:
    interl=[]
    liste_li=ul.findAll("li")
    for li in liste_li:
        try:
            interl.append(li.find("a").get("title"))
        except:
            pass
    auteurs1.append(interl)

#print(len(auteurs1[:len(auteurs1)-2]))  #On a bien les 26 lettres de l'alphabet
auteurs1=auteurs1[:len(auteurs1)-2]        #à la fin on a deux blocs hors sujet à enlever


liste_auteurs=[]        #liste avec tous les auteurs rassemblés
for autL in auteurs1:
    liste_auteurs+=autL
#print(len(liste_auteurs),liste_auteurs)
print(auteurs1[3])

['Abdoulaye Y. Dabo', 'Didier Daeninckx', 'Seyhmus Dagtekin', 'Alain Damasio', 'Daniel-Rops', 'Daniel Danis', 'Maurice G. Dantec', 'Pierre Daninos', 'Emile Danoën', 'Louis Dantin', 'Frédéric Dard', 'Milan Dargent', 'Guy Darol', 'Marie Darrieussecq', 'Léo Dartey', 'Simon Dary', 'Martine Dassault', 'Alphonse Daudet', 'Léon Daudet', 'René Daumal', 'Maurice David (résistant)', 'René David (auteur)', 'Mercedes Deambrosis', 'Max Deauville', 'Roland de Bodt', 'Daniel de Bruycker', 'Alain Decaux', 'Didier Decoin', 'Marien Defalvard', 'Laurent de Graeve', 'Jean-Marie Déguignet', 'Régine Deforges', 'Pierre Abraham de La Bretonnière', 'Jean-François de La Croix', 'Serge Delaive', 'Casimir Delavigne', 'Chloé Delaume', 'Philippe Delerm', 'Gilles Deleuze', 'Jeanne-Mance Delisle', 'Olivier Delorme', 'Geoffroy Delorme', 'Bernard Deloupy', 'Joseph Delteil (poète)', 'Dominique Demers', 'Alain Demouzon', 'Annick Demouzon', 'Jean-François Deniau', 'Patrick Denieul', 'Bantcheva Denitza', 'Maurice Denuzière

Première série de nettoyage : <br>
- on enlève les auteurs sans page qui sont dans la liste. Ex: <i> auteurs1[2] = [..., 'Catherine Chauchat (page inexistante)', ...] </i>
- certaines années apparaissent seules dans la liste, on les retire. Ex: <i> [...,'Hégésippe Moreau', '1962', 'Paul Morelle',...] </i>

In [26]:
for aut in liste_auteurs:
    if 'page inexistante' in aut:               #'Catherine Chauchat (page inexistante)'
        liste_auteurs.remove(aut)
    else:
        try:
            if float(aut).is_integer()==True:   #il y a des années seules dans la liste : [...,'Hégésippe Moreau', '1962', 'Paul Morelle',...]
                liste_auteurs.remove(aut)
        except:
            pass

Nettoyage de <code>liste_auteurs</code> avec les expressions régulières pour enlever les "(...)" <br>
On a par exemple : <i>[..., 'Denis Côté (écrivain)', ..., 'Maurice David (résistant)', 'René David (auteur)', ...]</i>

In [32]:
def regexpclean(text):
    if '(' in text:
        return re.sub(r"\([^()]*\)", "",text)
    else:
        return text

In [33]:
liste_auteurs=[regexpclean(aut) for aut in liste_auteurs]
print(len(liste_auteurs), liste_auteurs)

1554 ["François-Jean Willemain d'Abancourt", 'Jacques Abbadie', 'Éliette Abécassis', 'Pierre Abélard', 'Raymond Abellio', 'Edmond About', 'Pierre Abraham', 'Jean-Pierre Abraham', 'Laure Abrantès', 'Léon Abric', 'Amédée Achard', 'Marcel Achard', 'Paul Acker', 'Louise Ackermann', 'Germaine Acremant', 'Gérard Adam', 'Paul Adam', 'Adam de la Halle', 'Adam de Ros', 'André-Marcel Adamek', 'Adenet le Roi', 'Maylis Adhémar', 'Etel Adnan', 'Razika Adnani', 'Emmanuel Aegerter', 'Alain Agat', "Théodore Agrippa d'Aubigné", 'Dominique Aguessy', 'Jeannette D. Ahonsou', 'Jean Aicard', "D'Aigue d'Iffremont", 'Gustave Aimard', 'Charlotte Aïssé', 'Jean Ajalbert', 'Émile Ajar', 'Said Akl', 'Alain ', 'Alain de Lille', 'Alain-Fournier', 'Albéric de Pisançon', 'Guillaume Alexis', "Jean Le Rond d'Alembert", 'Alexandre de Bernay', 'Paul Alexis', 'François-Paul Alibert', 'Marcel Allain', 'Alphonse Allais', 'Ferdinand Alquié', 'Louis Althusser', 'Carlos Alvarado ', 'Jacques-Pierre Amée', 'Karim Amellal', 'Denis